In [5]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
# set this so LinearModel.fit won't have bug
tf.config.get_soft_device_placement()

True

#### Prepare data for model

In [6]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [7]:
# 将类别转为数字
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

# 将没用的列Drop
x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

# 将target单独拿出
y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [8]:
# Data setup for TensorFlow 1 with `tf.estimator`
def _input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
    feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
    feature_columns.append(feat_col)

In [9]:
# Create a method to instantiate a simplistic sample optimizer to use with our various TensorFlow 1 Estimator and TensorFlow 2 Keras models

def create_sample_optimizer(tf_version):
    if tf_version == 'tf1':
        optimizer = lambda: tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf1.train.exponential_decay(
                learning_rate=0.1,
                global_step=tf1.train.get_global_step(),
                decay_steps=10000,
                decay_rate=0.9))
    elif tf_version == 'tf2':
        optimizer = tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
    return optimizer


#### Create Model

In [ ]:
optimizer = create_sample_optimizer('tf1')

combined_estimator = tf.estimator.DNNLinearCombinedEstimator(
    head=tf.estimator.BinaryClassHead(),
    # Wide settings
    linear_feature_columns=feature_columns,
    linear_optimizer=optimizer,
    # Deep settings
    dnn_feature_columns=feature_columns,
    dnn_hidden_units=[128],
    dnn_optimizer=optimizer)

In [ ]:
combined_estimator.train(input_fn=_input_fn, steps=100)

In [ ]:
combined_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

In [10]:
# Create LinearModel and DNN Model as in Examples 1 and 2
optimizer = create_sample_optimizer('tf2')

# wide part
linear_model = tf.compat.v1.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10, verbose=100)

# deep part
dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])
dnn_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

Metal device set to: Apple M1 Pro
Epoch 1/10


2022-03-01 22:44:14.431356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-01 22:44:14.431495: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-01 22:44:14.482839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


InvalidArgumentError: Cannot assign a device for operation linear_model/dense/MatMul/ReadVariableOp: Could not satisfy explicit device specification '' because the node {{colocation_node linear_model/dense/MatMul/ReadVariableOp}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ResourceApplyFtrl: CPU 
Mul: GPU CPU 
AddV2: GPU CPU 
RealDiv: GPU CPU 
ReadVariableOp: GPU CPU 
Const: GPU CPU 
_Arg: GPU CPU 

Colocation member